### Columns for our Current Matching Model:

1. Last Name
2. First Name
3. State of Province
4. County
5. Place
6. Ward
7. Street
8. Place of Birth
9. Roll or Sheet
10. Latitude
11. Longitude
12. Census Year
13. Calculated Birth Year
14. Sex
15. Color or Ethnicity
16. Marital Status
17. Profession
18. Notable
19. Religion

### 1. Set Up

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import sys
sys.setrecursionlimit(1000000)

In [3]:
import pandas as pd
import numpy as np
import os

# Record Linkage
import recordlinkage as rl
from recordlinkage.index import Block
from recordlinkage.preprocessing import clean
from recordlinkage.preprocessing import phonetic

# Regular expression operations
import re

### 2. Load Data

In [4]:
current_master_df = pd.read_excel("current_master_6_17_2020.xlsx")

In [5]:
new_df = pd.read_excel('Rough version of all Tom Brooks.xlsx')

In [6]:
new_df

,LASTNAME,FIRSTNAME,REGIMENT,BIRTHPLACE,DATE OF ENLISTMENT,PLACE OF ENLISTMENT,AGE,OCCUPATION,NOTES,CAUSE OF DEATH,DATE OF DEATH,PLACE OF DEATH,PLACE OF BURIAL,SOURCE,ORDER NUMBER
0,"Albert, George Henry54th Mass Inf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"KIA July 18, 1863",NaN,"Fort Wagner, S.C.",NaN,Tom Brooks page 01,1.0
1,"Bailey, James 102nd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease Nov 20, 1864",NaN,"Beaufort, S.C. Beaufort National Cemetery, Sec...",NaN,Tom Brooks page 01,2.0
2,"Banks, Daniel 43rd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pneum'a March 16, 1865 Point of Rocks, Va",NaN,NaN,NaN,Tom Brooks page 01,3.0
3,"Benjamin, James 20th USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease Aug 22, 1865",NaN,"New Orleans, La Monument National Cemetery, Ch...",NaN,Tom Brooks page 01,4.0
4,"Benton, James 43rd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pneumonia Dec 7, 1864",NaN,"Point of Rocks, Va",NaN,Tom Brooks page 01,5.0
5,"Brown, Abraham 54th Mass Inf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Accid Shot July 7, 1863",NaN,"James Island, SC accidentally shot with his ow...",NaN,Tom Brooks page 01,6.0
6,"Campbell, John 62nd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Diarrhea March 1, 1864",NaN,"Port Hudson, La",NaN,Tom Brooks page 01,7.0
7,"Clayton, CharlesE 11th USCHA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease April 6, 1864",NaN,"Englishtown, La",NaN,Tom Brooks page 01,8.0
8,"Coleman, John W = 102nd USCT.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tuberlois July 2, 1864",NaN,"Beaufort, S.C. Beaufort National Cemetery, Sec...",NaN,Tom Brooks page 01,9.0
9,"Derling, Aron 31st USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease July 21, 1865",NaN,"Brownsville, Texas Brownsville National Cemete...",NaN,Tom Brooks page 01,10.0


In [7]:
new_df[pd.isna(new_df.FIRSTNAME)]

,LASTNAME,FIRSTNAME,REGIMENT,BIRTHPLACE,DATE OF ENLISTMENT,PLACE OF ENLISTMENT,AGE,OCCUPATION,NOTES,CAUSE OF DEATH,DATE OF DEATH,PLACE OF DEATH,PLACE OF BURIAL,SOURCE,ORDER NUMBER
0,"Albert, George Henry54th Mass Inf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"KIA July 18, 1863",NaN,"Fort Wagner, S.C.",NaN,Tom Brooks page 01,1.0
1,"Bailey, James 102nd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease Nov 20, 1864",NaN,"Beaufort, S.C. Beaufort National Cemetery, Sec...",NaN,Tom Brooks page 01,2.0
2,"Banks, Daniel 43rd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pneum'a March 16, 1865 Point of Rocks, Va",NaN,NaN,NaN,Tom Brooks page 01,3.0
3,"Benjamin, James 20th USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease Aug 22, 1865",NaN,"New Orleans, La Monument National Cemetery, Ch...",NaN,Tom Brooks page 01,4.0
4,"Benton, James 43rd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Pneumonia Dec 7, 1864",NaN,"Point of Rocks, Va",NaN,Tom Brooks page 01,5.0
5,"Brown, Abraham 54th Mass Inf",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Accid Shot July 7, 1863",NaN,"James Island, SC accidentally shot with his ow...",NaN,Tom Brooks page 01,6.0
6,"Campbell, John 62nd USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Diarrhea March 1, 1864",NaN,"Port Hudson, La",NaN,Tom Brooks page 01,7.0
7,"Clayton, CharlesE 11th USCHA",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease April 6, 1864",NaN,"Englishtown, La",NaN,Tom Brooks page 01,8.0
8,"Coleman, John W = 102nd USCT.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tuberlois July 2, 1864",NaN,"Beaufort, S.C. Beaufort National Cemetery, Sec...",NaN,Tom Brooks page 01,9.0
9,"Derling, Aron 31st USCT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Disease July 21, 1865",NaN,"Brownsville, Texas Brownsville National Cemete...",NaN,Tom Brooks page 01,10.0
